### 1.data load

In [143]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

#  data load
df2019 = pd.read_csv("./data/2019_사업보고서_01_재무상태표_연결_20200623.txt", sep = "\t", encoding = "cp949")

### 2. 불필요한 변수 제거

In [144]:
# 필요 없는 변수 제거
def delete_col(data):
    data.drop("통화", axis = 1, inplace = True)
    data.drop("Unnamed: 15", axis = 1, inplace = True)
    data.drop("전기", axis = 1, inplace = True)
    data.drop("전전기", axis = 1, inplace = True)
    data.drop("재무제표종류", axis = 1, inplace = True)
    return data

df2019 = delete_col(df2019)

### 3. 연속형변수(당기, 전기, 전전기)  dtype 변경

In [145]:
def str_to_float(data):
    import numpy as np
    
    data["당기"] = data["당기"].str.replace(",", "")
#     data["전기"] = data["전기"].str.replace(",", "")
#     data["전전기"] = data["전전기"].str.replace(",", "")
    data["당기"] = data["당기"].astype(np.float32)
    return data

df2019 = str_to_float(df2019)

### 4. 항목명 변수 처리

In [146]:
def duplication_check(data):
    # 항목코드 중복 확인
    entity_코드 = []
    entity_명 = []
    without_entity_코드 = []
    without_entity_명 = []

    # 각 리스트에 코드 및 코드명 append(소문자 처리)
    for idx in range(len(data)):
        if "entity" in data["항목코드"][idx]:
            entity_코드.append(data["항목코드"][idx].lower())
            entity_명.append(data["항목명"][idx].lower())
        else:
            without_entity_코드.append(data["항목코드"][idx].lower())
            without_entity_명.append(data["항목명"][idx].lower())

    # !entity 코드 딕셔너리
    dic_without_entity = {re.findall("_\w{1,}", without_entity_코드[idx])[0].lower():[] for idx in range(len(without_entity_코드))}

    for i, j in zip(without_entity_코드, without_entity_명):
        dic_without_entity[re.findall("_\w{1,}", i)[0].lower()].append(j)

    check_without_entity = {i:len(set(j)) for i, j in dic_without_entity.items()}
    
    # entity 코드 딕셔너리
    dic_entity = {entity_코드[idx]:[] for idx in range(len(entity_코드))}
    
    for i, j in zip(entity_코드, entity_명):
        dic_entity[i].append(j)
        
    check_entity = {i:len(set(j)) for i, j in dic_entity.items()}
    
    return entity_코드, entity_명, without_entity_코드, without_entity_명, dic_without_entity, dic_entity

entity_코드, entity_명, without_entity_코드, without_entity_명, dic_without_entity, dic_entity = duplication_check(df2019)

### 5. 공시된 항목명, 항목코드 가져오기(금융감독원 재무제표 양식)

In [147]:
def extract_element_id(data):
    element_idx_lst_entity = []
    element_idx_lst_without_entity = []
    
    for idx in range(len(data)):
        if "entity" in data["항목코드"][idx]:
            element_idx_lst_entity.append(data["항목코드"][idx])
        else:
            element_idx_lst_without_entity.append(data["항목코드"][idx])
            
    entity = sorted(set(element_idx_lst_entity), reverse = True)
    without_entity = sorted(set(element_idx_lst_without_entity), reverse = True)
    
    return entity, without_entity

a, b = extract_element_id(df2019)
pd.Series(b).to_csv("./2019항목명.csv", encoding = "euc-kr", index = False)

elementid = pd.read_excel("./data/재무제표양식.xlsx", encoding = "utf-8", sheet_name = "BS1")
# '한글 Label', 'Element ID'
df_element = elementid[['한글 Label', 'Element ID']]
df_element = df_element[df_element["Element ID"].notnull()]

df_element.columns = df_element.columns.str.replace(" ", "_")
lst_element = df_element["Element_ID"].unique()

re_lst_element = []
for i in lst_element:
    a = re.findall("_\w{1,}", i)[0].lower()
    re_lst_element.append(a)
 
dic_element = {i:[] for i in re_lst_element}
for i, j in zip(re_lst_element, df_element["한글_Label"]):
    dic_element[i].append(j)

### 6. entity 제외 항목명 변수 전처리

In [279]:
for idx in range(len(df2019)):
    if "entity" not in df2019["항목코드"][idx]:
        df2019["항목명"][idx] = dic_element[re.findall("_\w{1,}", df2019["항목코드"][idx])[0].lower()][0]

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


### 7. 정제된 데이터 프레임 만들기 

In [190]:
# DB 테이블에 적용할 D.F
# 회사이름 추출
lst_company = df2019["회사명"].unique()
element_value = []
for i in dic_element.values():
    element_value.append(i[0])
    
# columns
col = list(df2019.columns[0:8]) + element_value
check_col = list(dic_element.keys())

#  해당년도 데이터 행길이
company_dic = {i:[] for i in lst_company}
day = []

for i in lst_company:
    company_dic[i].append(df2019[df2019["회사명"] == i]["결산기준일"].unique())
    day.append(len(df2019[df2019["회사명"] == i]["결산기준일"].unique()))
    
row = sum(day)

bon = np.zeros([row,len(col)]) + np.NAN

# D.F
last_df = pd.DataFrame(bon, columns = col)

### 8. 데이터 입력

In [222]:
dic_standard = {i:list(df2019[df2019["회사명"] == i]["결산기준일"].unique()) for i in lst_company}
last_company = []
last_standard  = []
for key, value in dic_standard.items():
    for index in range(len(value)):
        last_company.append(key)
        last_standard.append(value[index])
        
cnt = 0

for idx in range(len(last_company)):

    a = df2019[(df2019["회사명"] == last_company[idx]) & (df2019["결산기준일"] == last_standard[idx])]
    a.reset_index(drop = True, inplace = True)
    for i in range(8):
        last_df.iloc[cnt, i] = a.iloc[0, i]

    for idx3 in range(len(a)):
        if "entity" not in a["항목코드"][idx3]:
            loc = dic_element[re.findall("_\w{1,}", a["항목코드"][idx3])[0].lower()][0]
            last_df.loc[cnt, loc] = a["당기"][idx3]
    cnt += 1

### 9. 2018데이터 입력

In [260]:
# 2018년 변수 처리
df = pd.read_csv("./data/2019_사업보고서_01_재무상태표_연결_20200623.txt", sep = "\t", encoding = "cp949")

def str_to_float(data):
    import numpy as np
    
    data["당기"] = data["당기"].str.replace(",", "")
    data["전기"] = data["전기"].str.replace(",", "")
    data["전전기"] = data["전전기"].str.replace(",", "")
    data["당기"] = data["당기"].astype(np.float32)
    data["전기"] = data["전기"].astype(np.float32)
    data["전전기"] = data["전전기"].astype(np.float32)
    return data

df = str_to_float(df)

# 필요 없는 변수 제거
def delete_col(data):
    data.drop("통화", axis = 1, inplace = True)
    data.drop("Unnamed: 15", axis = 1, inplace = True)
    data.drop("당기", axis = 1, inplace = True)
    data.drop("전전기", axis = 1, inplace = True)
    data.drop("재무제표종류", axis = 1, inplace = True)
    return data

df = delete_col(df)

# DB 테이블에 적용할 D.F
# 회사이름 추출
lst_company = df2019["회사명"].unique()
element_value = []
for i in dic_element.values():
    element_value.append(i[0])
    
# columns
col = list(df.columns[0:8]) + element_value
check_col = list(dic_element.keys())

#  해당년도 데이터 행길이
company_dic = {i:[] for i in lst_company}
day = []

for i in lst_company:
    company_dic[i].append(df[df["회사명"] == i]["결산기준일"].unique())
    day.append(len(df[df["회사명"] == i]["결산기준일"].unique()))
    
row = sum(day)

bon = np.zeros([row,len(col)]) + np.NAN

# D.F
add_df = pd.DataFrame(bon, columns = col)


dic_standard = {i:list(df[df["회사명"] == i]["결산기준일"].unique()) for i in lst_company}
last_company = []
last_standard  = []
for key, value in dic_standard.items():
    for index in range(len(value)):
        last_company.append(key)
        last_standard.append(value[index])
        
cnt = 0

for idx in range(len(last_company)):

    a = df[(df["회사명"] == last_company[idx]) & (df["결산기준일"] == last_standard[idx])]
    a.reset_index(drop = True, inplace = True)
    for i in range(8):
        add_df.iloc[cnt, i] = a.iloc[0, i]

    for idx3 in range(len(a)):
        if "entity" not in a["항목코드"][idx3]:
            loc = dic_element[re.findall("_\w{1,}", a["항목코드"][idx3])[0].lower()][0]
            add_df.loc[cnt, loc] = a["전기"][idx3]
    cnt += 1

add_df["결산기준일"] = add_df["결산기준일"].str.replace("2019", "2018")

last_df = pd.concat([last_df, add_df]).reset_index(drop = True)

### 10. 2017 데이터 입력

In [262]:
# 2017년 변수 처리
df = pd.read_csv("./data/2019_사업보고서_01_재무상태표_연결_20200623.txt", sep = "\t", encoding = "cp949")

def str_to_float(data):
    import numpy as np
    
    data["당기"] = data["당기"].str.replace(",", "")
    data["전기"] = data["전기"].str.replace(",", "")
    data["전전기"] = data["전전기"].str.replace(",", "")
    data["당기"] = data["당기"].astype(np.float32)
    data["전기"] = data["전기"].astype(np.float32)
    data["전전기"] = data["전전기"].astype(np.float32)
    return data

df = str_to_float(df)

# 필요 없는 변수 제거
def delete_col(data):
    data.drop("통화", axis = 1, inplace = True)
    data.drop("Unnamed: 15", axis = 1, inplace = True)
    data.drop("당기", axis = 1, inplace = True)
    data.drop("전기", axis = 1, inplace = True)
    data.drop("재무제표종류", axis = 1, inplace = True)
    return data

df = delete_col(df)

# DB 테이블에 적용할 D.F
# 회사이름 추출
lst_company = df2019["회사명"].unique()
element_value = []
for i in dic_element.values():
    element_value.append(i[0])
    
# columns
col = list(df.columns[0:8]) + element_value
check_col = list(dic_element.keys())

#  해당년도 데이터 행길이
company_dic = {i:[] for i in lst_company}
day = []

for i in lst_company:
    company_dic[i].append(df[df["회사명"] == i]["결산기준일"].unique())
    day.append(len(df[df["회사명"] == i]["결산기준일"].unique()))
    
row = sum(day)

bon = np.zeros([row,len(col)]) + np.NAN

# D.F
add_df = pd.DataFrame(bon, columns = col)


dic_standard = {i:list(df[df["회사명"] == i]["결산기준일"].unique()) for i in lst_company}
last_company = []
last_standard  = []
for key, value in dic_standard.items():
    for index in range(len(value)):
        last_company.append(key)
        last_standard.append(value[index])
        
cnt = 0

for idx in range(len(last_company)):

    a = df[(df["회사명"] == last_company[idx]) & (df["결산기준일"] == last_standard[idx])]
    a.reset_index(drop = True, inplace = True)
    for i in range(8):
        add_df.iloc[cnt, i] = a.iloc[0, i]

    for idx3 in range(len(a)):
        if "entity" not in a["항목코드"][idx3]:
            loc = dic_element[re.findall("_\w{1,}", a["항목코드"][idx3])[0].lower()][0]
            add_df.loc[cnt, loc] = a["전전기"][idx3]
    cnt += 1

add_df["결산기준일"] = add_df["결산기준일"].str.replace("2019", "2017")

last_df = pd.concat([last_df, add_df]).reset_index(drop = True)

In [264]:
last_df.to_csv("./data/2017_2019_without_entity.csv", encoding = "euc-kr", index = False)

In [281]:
df2019[(df2019["회사명"] == "리드코프") & (df2019["항목명"] == '      차입부채')]

,종목코드,회사명,시장구분,업종,업종명,결산월,결산기준일,보고서종류,항목코드,항목명,전기
23746,[012700],리드코프,코스닥시장상장법인,467,기타 전문 도매업,12,2019-12-31,사업보고서,entity00117708_udf_BS_202031793352138_udf_BS_2...,차입부채,7.363806e+11


In [282]:
last_df[last_df["회사명"] == "3S"]

,종목코드,회사명,시장구분,업종,업종명,결산월,결산기준일,보고서종류,재무상태표 [abstract],자산 [abstract],...,현금흐름위험회피파생상품평가손실,지분법기타포괄손익변동,이익잉여금(결손금),이익준비금,기타법정적립금,임의적립금,미처분이익잉여금(미처리결손금),비지배지분,자본총계,자본과부채총계
0,[060310],3S,코스닥시장상장법인,292.0,특수 목적용 기계 제조업,3.0,2019-03-31,사업보고서,NaN,NaN,...,NaN,NaN,-3.167278e+10,NaN,NaN,NaN,NaN,0.0,3.437168e+10,5.332922e+10
1692,[060310],3S,코스닥시장상장법인,292.0,특수 목적용 기계 제조업,3.0,2018-03-31,사업보고서,NaN,NaN,...,NaN,NaN,-3.236181e+10,NaN,NaN,NaN,NaN,0.0,3.075234e+10,4.952100e+10
3384,[060310],3S,코스닥시장상장법인,292.0,특수 목적용 기계 제조업,3.0,2017-03-31,사업보고서,NaN,NaN,...,NaN,NaN,-3.092958e+10,NaN,NaN,NaN,NaN,0.0,2.917967e+10,5.370690e+10


In [287]:
df.to_csv("./data/2019_2017변수확인.csv", index = False, encoding = "euc-kr")